In [1]:
#library
!pip install numpy pandas scipy shapely scikit-image

In [2]:
# kernel
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
from shapely.geometry import Polygon
from scipy import ndimage
from skimage import measure

def compute_kde_area(df, selected_codes, percentile=95, grid_res=100j, bandwidth=0.2):
    """
    Compute KDE-based area estimation (in m²) for a list of codes using UTM coordinates.
    
    Parameters:
        df (DataFrame): Input DataFrame with columns 'code', 'x_utm', 'y_utm'.
        selected_codes (list): List of codes to process.
        percentile (float): Percentile threshold for KDE contour (e.g., 95).
        grid_res (complex): Grid resolution (e.g., 100j).
        bandwidth (float): Bandwidth for KDE.

    Returns:
        DataFrame: Code-indexed DataFrame with KDE Area in m².
    """
    results = []

    for code in selected_codes:
        code_data = df[df['Code'] == code].dropna(subset=['UTM_E', 'UTM_N'])
        coords = np.vstack((code_data['UTM_E'], code_data['UTM_N'])).T

        if len(coords) < 10:
            results.append({'Code': code, 'KDE Area (m²)': np.nan})
            continue

        try:
            kde = gaussian_kde(coords.T, bw_method=bandwidth)
        except np.linalg.LinAlgError:
            results.append({'Code': code, 'KDE Area (m²)': np.nan})
            continue

        x_min, x_max = coords[:, 0].min() - 5, coords[:, 0].max() + 5
        y_min, y_max = coords[:, 1].min() - 5, coords[:, 1].max() + 5

        xgrid, ygrid = np.mgrid[x_min:x_max:grid_res, y_min:y_max:grid_res]
        grid_coords = np.vstack([xgrid.ravel(), ygrid.ravel()])
        density = np.reshape(kde(grid_coords), xgrid.shape)

        level = np.percentile(density, percentile)
        contours = measure.find_contours(density, level)

        max_area = 0
        for contour in contours:
            poly_coords = [
                (
                    x_min + (x_max - x_min) * p[1] / density.shape[1],
                    y_min + (y_max - y_min) * p[0] / density.shape[0]
                )
                for p in contour
            ]
            if len(poly_coords) >= 3:
                polygon = Polygon(poly_coords)
                if polygon.is_valid:
                    max_area = max(max_area, polygon.area)

        results.append({'Code': code, 'KDE Area (m²)': max_area if max_area > 0 else np.nan})

    return pd.DataFrame(results).set_index('Code')


In [3]:
# df deve contenere le colonne: 'code', 'x_utm', 'y_utm'
df=pd.read_csv("NaaVREcourse/ant_coord.csv", sep=",")

kde_results = compute_kde_area(df, selected_codes=['A1', 'B2'], percentile=95)
print(kde_results)


      KDE Area (m²)
Code               
A1              NaN
B2              NaN
